In [1]:
import pandas as pd
import matplotlib.pyplot as plt


### The pandas `.concat( )`  method is used to concatenate two dataframes based on shared column names
- we will create two dataframes and then concatenate them
- this would be useful if you had, for example, two different years of data with the same columns that you want to explore together; in a case like this, you would want to add a year column before joining them


In [4]:
data_2023 = { 'state' : ['NY','WV','TN'],
         'count' : [14,55,63],
         'code' : ['ab65','88ui','u7r4']}
data_2024 = { 'state' : ['NY','WV','TN'],
         'count' : [7,78,29],
         'code' : ['0o09','jj33','1d6u']}

In [6]:
df_2023 = pd.DataFrame(data_2023)
df_2024 = pd.DataFrame(data_2024)

In [8]:
df_2023

,state,count,code
0,NY,14,ab65
1,WV,55,88ui
2,TN,63,u7r4


In [10]:
df_2024

,state,count,code
0,NY,7,0o09
1,WV,78,jj33
2,TN,29,1d6u


#### Before we concatenate `df_2023` and `df_2024` we need to add a column for the year

In [13]:
df_2023['year'] = '2023'
df_2024['year'] = '2024'

In [15]:
df_2023

,state,count,code,year
0,NY,14,ab65,2023
1,WV,55,88ui,2023
2,TN,63,u7r4,2023


In [17]:
concat_data = pd.concat([df_2023, df_2024])
concat_data

## Notice that the index repeats itself

,state,count,code,year
0,NY,14,ab65,2023
1,WV,55,88ui,2023
2,TN,63,u7r4,2023
0,NY,7,0o09,2024
1,WV,78,jj33,2024
2,TN,29,1d6u,2024


### The `reset_index( )` method will change these to a 0-based incrementing index
- add the `drop = True` argument to prevent saving the current index as a column


In [20]:
concat_data = concat_data.reset_index(drop = True)
print(concat_data)

  state  count  code  year
0    NY     14  ab65  2023
1    WV     55  88ui  2023
2    TN     63  u7r4  2023
3    NY      7  0o09  2024
4    WV     78  jj33  2024
5    TN     29  1d6u  2024


### Another method for combining data is  `merge( )`
- First we'll read in the schools data again
- And we'll read in the school indicator scores


In [23]:
schools = pd.read_csv('../data/schools_clean.csv')
schools.head(2)

,level,name,zipcode,grade_k,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,...,hisp,p_islander,white,male,female,econ_disadv,disabled,limited_eng,lat,lng
0,Elementary School,A. Z. Kelley Elementary,37013,153.0,145.0,149.0,180.0,184.0,NaN,NaN,...,206,1.0,212.0,431,421,261,75.0,298.0,36.021817,-86.658848
1,Elementary School,Alex Green Elementary,37189,42.0,50.0,44.0,38.0,24.0,NaN,NaN,...,29,1.0,21.0,115,119,153,21.0,25.0,36.252961,-86.832229


In [25]:
indicators = pd.read_csv('../data/school_indicator_scores_suppressed.csv')
indicators.head(2)

,system,system_name,school,school_name,pool,designation_ineligible,subgroup,score_achievement,score_growth,score_absenteeism,score_grad,score_ready_grad,score_elpa
0,10,Anderson County,2,Anderson County High School,HS,0.0,All Students,3.0,4.0,4.0,4.0,4.0,NaN
1,10,Anderson County,2,Anderson County High School,HS,0.0,American Indian or Alaska Native,NaN,NaN,NaN,NaN,NaN,NaN


### Before we merge, we need to decide 
- which columns to keep from each dataframe  
- what _type_ of merge we want

#### Let's say we want to look at the effect of poverty on school indicators
- From the schools data, we need name, total number of students, and total who are economically disadvantaged
    - we need to create a column for the total number of students before we can subset the schools dataframe
- From the school indicators dataframe, we need school name, and the six *score* columns
    - filtering the dataframe first so that we only look at Davidson County schools is a good idea
    - we also want to filter in order to keep only the rows where subgroup is "All Students"


In [ ]:
schools['total_students'] = schools.male + schools.female
schools.head(2)

In [ ]:
schools_subset = schools[['name', 'total_students', 'econ_disadv']]

In [ ]:
indicators = indicators.loc[(indicators.system_name == 'Davidson County') & (indicators.subgroup == 'All Students')]
indicators.shape

In [ ]:
indicators.head(3)

In [ ]:
indicators_subset = indicators[['school_name', 'score_achievement', 
                                'score_growth', 'score_absenteeism',
                                'score_grad', 'score_ready_grad', 
                                'score_elpa']]

In [ ]:
print('schools subset: ', schools_subset.shape)
print('indicators subset: ', indicators_subset.shape)

### There are 12 more rows in the schools_subset data than there are in the indicators_subset
- We want data that matches to **both** datasets 
![pandas merge types](../images/pandas_merge_types.png)
- So we want an inner join (which is the default)

In [ ]:
schools_with_scores = pd.merge(schools_subset, indicators_subset, 
                               left_on = 'name', right_on = 'school_name', 
                               how = 'inner')
schools_with_scores.head(3)

### We can drop one of the school name columns and calculate the percentage of students from each school who are economically disadvantaged.

In [ ]:
schools_with_scores = schools_with_scores.drop(columns = 'school_name')

In [ ]:
schools_with_scores['pct_econ_disadv'] = schools_with_scores.econ_disadv / schools_with_scores.total_students * 100
schools_with_scores.head()

### We'll take a quick peak at the relationship between the percentage of students who are economically disadvantaged and the achievement score  for schools and then save this dataset for further exploration

In [ ]:
plt.scatter(x = 'score_achievement', y ='pct_econ_disadv', data = schools_with_scores)
plt.xlabel('score')
plt.ylabel('percentage economically disadvantaged')
plt.title('Score by percent economically disadvantaged')

In [ ]:
schools_with_scores.to_csv('../data/schools_with_scores.csv', index = False)

In [ ]:
clean = pd.read_csv('../data/schools_with_scores.csv')
clean.head()

# End of Instruction

### You want to plot the score distribution by zipcode for all zipcodes that have 10 or more schools.

In [ ]:
### First, slice the schools dataframe so you only have the 'name' and 'zipcode' columns.  Save this as zips.

In [ ]:
zips = schools[['name','zipcode']]

In [ ]:
zips.head()

In [ ]:
### Next, merge zips with_with_scores and name this dataframe test_zips

In [ ]:
test_zips = pd.merge(zips, schools_with_scores, 
                               left_on = 'name', right_on = 'name', 
                               how = 'inner')

In [ ]:
### Now use value_counts() to find which zipcodes have 10 or more schools.
### (You can simply make a note, later in the course we learn how to automate this step)

In [ ]:
test_zips.zipcode.value_counts()

In [ ]:
### Filter the test_zips dataframe to include only those zipcodes that contained 10 or more schools

In [ ]:
list  = [ 37013, 37211, 37207, 37209]
test_zips = test_zips.loc[test_zips.zipcode.isin(list)]


In [ ]:
### Create a scatterplot that shows the distribution of scores for those zipcodes.

In [ ]:
plt.scatter( x= 'zipcode', y = 'score_grad', data= test_zips)